##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Image Augmentation Tutorial

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/image_augmentation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/image_augmentation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/image_augmentation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/image_augmentation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

This tutorial explains how to augment images using [tf.image](https://www.tensorflow.org/api_docs/python/tf/image), and training a [keras model](https://www.tensorflow.org/guide/keras/overview) using the augmented data.

## Setup

First, you will import `tensorflow`, along with `tensorflow_datasets`. You will use images from a tensorflow dataset for augmentation.

Next, import `keras` from `tensorflow`. This will be used for training a model with the augmented images.

Finally, import `pyplot` from `matplotlib` to show our unaugmented and augmented images.

In [0]:
from tensorflow import keras
from matplotlib import pyplot
import tensorflow_datasets as tfds
import tensorflow as tf
import random

### Load in the fashion dataset, and pre process the images. Then, show 9 unaugmented images.

You will first load in a dataset (in this case the fashion dataset). You will then pre-process the images a little bit, then show 9 of the unaugmented images.

In [0]:
# Load in dataset
dataset = keras.datasets.fashion_mnist
(training_images, training_labels), (testing_images, testing_labels) = dataset.load_data()

unaltered_images = training_images

# Pre-process the images
training_images = training_images / 255.0
unaltered_images = unaltered_images / 255.0
testing_images = testing_images / 255.0

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for i in range(0, 9):
	pyplot.subplot(330 + 1 + i)
	pyplot.tight_layout()
	pyplot.xlabel(class_names[training_labels[i]])
	pyplot.imshow(training_images[i], cmap=pyplot.get_cmap('gray'))

### Augment the images
Now you will augment the images, and show 9 of the augmented images.


---



#### Importance of augmentation
Augmentation is very important, because it can help improve the generalizability of our model. It also helps reduce over fitting our data. By artificially augmenting images, you are exposing our model to more data, which helps it detect much more than it would if the data was not augmented.

In [0]:
# Prepare images for augmentation
training_images = training_images.reshape((training_images.shape[0], 28, 28, 1))
training_images = training_images.astype('float32')

# Prepare images for augmentation
unaltered_images = unaltered_images.reshape((unaltered_images.shape[0], 28, 28, 1))
unaltered_images = unaltered_images.astype('float32')

# Prepare images for testing model
testing_images = testing_images.reshape((testing_images.shape[0], 28, 28, 1))
testing_images = testing_images.astype('float32')


# Augment the images with tf.image
def augment_image(x: tf.Tensor) -> tf.Tensor:
  # Changes the brightness to a random value between 0.1 and 0.9
  new_x = tf.image.adjust_brightness(x, random.uniform(0.1, 0.9))

  # Changes the contrast to a random value between 0.1 and o.9
  new_x = tf.image.adjust_contrast(images=new_x, contrast_factor=random.uniform(0.1, 0.9))

  # Rotate the image 0, 90, 180, 270, or 360 degrees, chosen randomly
  new_x = tf.image.rot90(new_x, k=random.randrange(0, 4))

  # Return the augmented image
  return new_x

# Loop through and augment the training images
for index, image in enumerate(training_images):
		training_images[index] = augment_image(image)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# Create a grid of 9 images
for i in range(0, 9):
	pyplot.subplot(330 + 1 + i)
	pyplot.tight_layout()
	pyplot.xlabel(class_names[training_labels[i]])
	pyplot.imshow(training_images[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
# Show the images
pyplot.show()

### Build and train a keras model
Now, train a keras model with the augmented images. First, create and compile the model, then train it with your augmented images.

In [0]:
def retrieve_model():
  # Prepare the layers for training with keras
  model = keras.Sequential([
      keras.layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      keras.layers.MaxPooling2D((2, 2)),
      keras.layers.Conv2D(56, (3, 3), activation='relu'),
      keras.layers.MaxPooling2D((2, 2)),
      keras.layers.Conv2D(56, (3, 3), activation='relu'),

      keras.layers.Flatten(),
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dense(10, activation='softmax')
  ])

  return model

model = retrieve_model()
# Compile the model for training with keras
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

unaugmented_model = retrieve_model()
# Compile the model for training with keras
unaugmented_model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with keras
model.fit(training_images, training_labels, epochs=5, validation_data=(testing_images, testing_labels))

# Train the model with unaugmented images
unaugmented_model.fit(unaltered_images, training_labels, epochs=5, validation_data=(testing_images, testing_labels))

### Evaluate the augmented and unaugmented models

In [0]:
# Evaluate the model on unaugmented images

# Prepare images for evaluation
testing_images = testing_images.reshape((testing_images.shape[0], 28, 28, 1))
testing_images = testing_images.astype('float32')

result = model.evaluate(testing_images, testing_labels, batch_size=128)
print('(AUGMENTED) Test loss, accuracy: ' + str(result))

unaugmented_result = unaugmented_model.evaluate(testing_images, testing_labels, batch_size=128)
print('(UNAUGMENTED) Test loss, accuracy: ' + str(unaugmented_result))

### Results
The results, contrary to what you may expect, show that the unaugmented images produced a slightly better result.

In this tutorial, you augmented the images quite a bit (i.e. the brightness and contrast were set between very low and very high ranges). In practice, you will not want to augment your images as much. Small augmentation will produce a more accurate model than heavy augmentation, like the one in this tutorial.